### Machine Learning Workshop example: Cats vs dogs

This notebook shows how to train a convolutional neural network that distinguishes cats and dogs. \
For more information on image classification, check out this blog: [How to Classifiy Photos of Dogs and Cats (with 97% accuracy)](https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-photos-of-dogs-and-cats/).

### Setup

Dataset is available on [Kaggle](https://www.kaggle.com/c/dogs-vs-cats) - download and extract a folder called `train/` in your working directory.  

- create a conda environment with required packages (in console): `conda create -n ml_workshop2 tensorflow-gpu` \
This requires your GPU to be CUDA-enabled - you may have to install additional drivers depending on your hardware.
- alternative: train using CPU (much slower): `conda create -n ml_workshop2 tensorflow`
- Install remaining libraries:

        conda activate ml_workshop2
		conda install keras scikit-learn ipykernel pillow
        python -m ipykernel install --user --name ml_workshop2 --display-name "ML Workshop - cats vs. dogs"
- run in local browser: `jupyter notebook`
- Alternative: run in VSCode with Python and Jupyter notebook extensions


In [10]:
# load dogs vs cats dataset 
from os import listdir
from numpy import asarray
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
import random

random.seed(42)

# define location of dataset
folder = 'train/'
X, y = list(), list()
image_size = (224, 224)
# enumerate files in the directory
progress = 0
for file in listdir(folder):
    if progress % 2500 == 0:
        print(progress / 250, r"% complete")
    
    # determine class based on file name
    output = 0.0
    if file.startswith('cat'):
        output = 1.0

    photo = load_img(folder + file, target_size=image_size)

    # convert to numpy array
    photo = img_to_array(photo)
    X.append(photo)
    y.append(output)

    progress += 1
    
# convert to a numpy arrays
X = asarray(X)
y = asarray(y)
print(X.shape, y.shape)

0.0 % complete
10.0 % complete
20.0 % complete
30.0 % complete
40.0 % complete
50.0 % complete
60.0 % complete
70.0 % complete
80.0 % complete
90.0 % complete
(25000, 224, 224, 3) (25000,)


In [11]:
# resampling using simple train/test split
import tensorflow
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [12]:
# we are using a pre-trained image classification model here and fine-tuning on our dataset
# for more information about the general procedure, see https://en.wikipedia.org/wiki/Transfer_learning
# for information about the base model, see https://keras.io/api/applications/vgg/

def define_transfer_learning_model():
    # load model
    model = tensorflow.keras.applications.VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = tensorflow.keras.layers.Flatten()(model.layers[-1].output)
    class1 = tensorflow.keras.layers.Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = tensorflow.keras.layers.Dense(1, activation='sigmoid')(class1)
    # define new model
    model = tensorflow.keras.Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = tensorflow.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

transfer_learned_model = define_transfer_learning_model()

In [13]:
# bit of black magic - data is centered using the same method as in the original training of the VGG16 model
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(featurewise_center=True)
datagen.mean = [123.68, 116.779, 103.939]

# dataset does not fit into the memory of my GPU - load data in batches
train_generator = datagen.flow(X_train, y_train)
test_generator = datagen.flow(X_test, y_test)

transfer_learned_model.fit(train_generator, steps_per_epoch=len(train_generator), validation_data=test_generator, validation_steps=len(test_generator), epochs=2, verbose=1)

Epoch 1/2
625/625 [==============================] - 155s 243ms/step - loss: 0.2066 - accuracy: 0.9661 - val_loss: 0.0636 - val_accuracy: 0.9714
Epoch 2/2
625/625 [==============================] - 168s 269ms/step - loss: 0.0245 - accuracy: 0.9912 - val_loss: 0.0640 - val_accuracy: 0.9762


In [14]:
import numpy as np

# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, target_size=image_size)
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 3 channels
	img = img.reshape(1, *image_size, 3)
	# center pixel data
	img = img.astype('float32')
	img = img - [123.68, 116.779, 103.939]
	return img

for idx in range(1, 7):
    pic = load_image(f"saxi_ari_photos/saxi{idx}.jpg")
    pred = transfer_learned_model.predict(pic, verbose=True)[0][0]
    print(f"Predicted probability of cat image #{idx} showing a cat: {round(pred, 2)}")

for idx in range(1, 5):
    pic = load_image(f"saxi_ari_photos/arinka{idx}.jpg")
    pred = transfer_learned_model.predict(pic, verbose=True)[0][0]
    print(f"Predicted probability of dog image #{idx} showing a cat: {round(pred, 2)}")

1/1 [==============================] - 0s 223ms/step
Predicted probability of cat image #1 showing a cat: 1.0
1/1 [==============================] - 0s 16ms/step
Predicted probability of cat image #2 showing a cat: 1.0
1/1 [==============================] - 0s 31ms/step
Predicted probability of cat image #3 showing a cat: 1.0
1/1 [==============================] - 0s 18ms/step
Predicted probability of cat image #4 showing a cat: 0.9300000071525574
1/1 [==============================] - 0s 16ms/step
Predicted probability of cat image #5 showing a cat: 0.9700000286102295
1/1 [==============================] - 0s 13ms/step
Predicted probability of cat image #6 showing a cat: 0.8799999952316284
1/1 [==============================] - 0s 36ms/step
Predicted probability of dog image #1 showing a cat: 0.0
1/1 [==============================] - 0s 32ms/step
Predicted probability of dog image #2 showing a cat: 0.0
1/1 [==============================] - 0s 16ms/step
Predicted probability of dog i